In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import liana as li
import anndata
import scanpy as sc
import os 
import scipy

In [2]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from utils import *

1.13.1+cu117
1.13.1+cu117


In [3]:
import argparse
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Entities
from torch_geometric.nn import FastRGCNConv, RGCNConv, GCNConv, InnerProductDecoder, GAE, VGAE
from torch_geometric.utils import k_hop_subgraph

from torch_geometric.datasets import Planetoid
from torch_geometric.data.data import Data
from torch_geometric.transforms import NormalizeFeatures


from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering
import random
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity

from scipy.io import mmread

from scipy.stats import wilcoxon
import anndata
import scanpy as sc

## Processing with csv file

In [13]:
matrix = pd.read_csv("../../data/GraphComm_Input/intercell_Drosophila/matrix.csv",index_col=0)

In [14]:
adata = anndata.AnnData(matrix.transpose())

In [15]:
adata

AnnData object with n_obs × n_vars = 5835 × 8443

In [16]:
sc.pp.filter_genes(adata, min_cells=5)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.neighbors(adata)
sc.tl.leiden(adata)

         Falling back to preprocessing with `sc.pp.pca` and default params.


In [17]:
meta = pd.DataFrame({"cell":adata.obs["leiden"].index.tolist(),"labels":adata.obs["leiden"].tolist()})

In [18]:
meta.index = meta["cell"].tolist()

In [19]:
adata.obs = meta

In [20]:
matrix = pd.DataFrame(adata.X.transpose(),columns=adata.obs.index.tolist(),index=adata.var.index.tolist())

# Processing with h5ad

In [12]:

adata = sc.read_h5ad("/data/raw_data/Cardiac_cells/Visium-RZ_FZ_P5.h5ad")
matrix = pd.DataFrame.sparse.from_spmatrix(adata.X,index=adata.obs.index.tolist(),columns=adata.var["feature_name"].tolist())
sc.pp.filter_genes(adata, min_cells=10)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.neighbors(adata)
sc.tl.leiden(adata)
matrix = pd.DataFrame.sparse.from_spmatrix(adata.X,index=adata.obs.index.tolist(),columns=adata.var["feature_name"].tolist())
meta = pd.DataFrame({"cell":adata.obs.index.tolist(),"labels":adata.obs["cell_type_original"].tolist()})
meta.index = meta["cell"].tolist()
#removing lymphoid as it is only one cell, cannot be used in benchmarking
meta = meta[meta['labels'] != "Lymphoid"]
adata = adata[meta["cell"].tolist()]
matrix = matrix.transpose()

matrix = matrix[meta.index.tolist()]

index = matrix.index.tolist()
adata.obs = meta


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/h/soemily/new_GAT/data/raw_data/Cardiac_cells/Visium-RZ_FZ_P5.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

# rest of preprocessing

In [21]:
#sc.tl.rank_genes_groups(adata, 'labels')
cell_groups = meta['labels'].unique().tolist()
matrix_list = {}
for i in cell_groups:
    cells = meta[meta["labels"]==i].index.tolist()
    temp_matrix = matrix[cells]
    matrix_list[i] = (temp_matrix.mean(axis=1)[temp_matrix.mean(axis=1) >= 0].index.tolist())

In [22]:
adata = adata.transpose()

In [23]:
cell_type_df = matrix_list

In [24]:
#mean_expression = np.mean(matrix.values[matrix.values != 0])
nodes = pd.DataFrame({"category":[],"identifier":[]})

In [25]:
LR_nodes = pd.read_csv("/data/LR_database/intercell_nodes.csv",index_col=0)
Omnipath_network = pd.read_csv("/data/LR_database/intercell_interactions.csv",index_col=0)

In [26]:
ligands = LR_nodes[LR_nodes["category"]=="Ligand"]["identifier"].tolist()
receptors = LR_nodes[LR_nodes["category"]=="Receptor"]["identifier"].tolist()

In [27]:
ligand_list = []
receptor_list = []
new_cell_df = {}
for i in cell_type_df.keys():
    ligand_list.extend(list(set(ligands) & set(cell_type_df[i])))
    receptor_list.extend(list(set(receptors) & set(cell_type_df[i])))
    new_cell_df[i] = [list(set(ligands) & set(cell_type_df[i])),list(set(receptors) & set(cell_type_df[i]))]

In [28]:
for i in new_cell_df.keys():
    new_cell_df[i][0] = [j+"_Ligand" for j in new_cell_df[i][0]]
    new_cell_df[i][1] = [j+"_Receptor" for j in new_cell_df[i][1]]

In [29]:
ligand_list = list(set(ligand_list))
receptor_list = list(set(receptor_list))

In [30]:
ligand_list = [i+"_Ligand" for i in ligand_list]
receptor_list = [i+"_Receptor" for i in receptor_list]

In [31]:
nodes = pd.concat([nodes,pd.DataFrame({"category":["Cell Group"] * len(list(cell_type_df.keys())), "identifier":list(cell_type_df.keys())})])
nodes = pd.concat([nodes,pd.DataFrame({"category":["Ligand"] * len(ligand_list), "identifier":ligand_list})])
nodes = pd.concat([nodes,pd.DataFrame({"category":["Receptor"] * len(receptor_list), "identifier":receptor_list})])

In [32]:
new_identifier = [row["identifier"] + "_" + row["category"] for index,row in LR_nodes.iterrows()]

In [33]:
LR_nodes["identifier"] = new_identifier

In [34]:
nodes["Id"] = range(0,nodes.shape[0])
nodes = nodes[["Id","category","identifier"]]

In [35]:
nodes.index = nodes.index.astype('int')
nodes["Id"] = nodes["Id"].astype('int')

In [36]:
meta.index = meta["cell"].tolist()

In [38]:
interactions = pd.DataFrame({"Src":[],"Dst":[],"Weight":[],"edge_type":[]})

In [39]:
LR_nodes.index = LR_nodes["Id"].tolist()

In [42]:
LR_nodes

,Id,category,identifier
1,1,Ligand,EPOR_Ligand
2,2,Ligand,FYN_Ligand
3,3,Ligand,KL_Ligand
4,4,Ligand,S100A10_Ligand
5,5,Ligand,JAG2_Ligand
...,...,...,...
1955,1955,Receptor,PRKCD_Receptor
1956,1956,Receptor,SLK_Receptor
1957,1957,Receptor,STK38_Receptor
1958,1958,Receptor,STK11_Receptor


In [44]:
Omnipath_network["Src"] += 1
Omnipath_network["Dst"] += 1

In [45]:
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["identifier"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["identifier"].tolist()


In [46]:
interactions

,Src,Dst,Weight,edge_type


In [47]:
source_list = []
dest_list = []
weight_list = []
edge_type_list = []
for i in new_cell_df.keys():
    source_list.extend([i] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
    dest_list.extend(new_cell_df[i][0])
    dest_list.extend(new_cell_df[i][1])
    weight_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
    edge_type_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))

In [48]:
interactions["Src"] = source_list
interactions["Dst"] = dest_list
interactions["Weight"] = weight_list
interactions["edge_type"] = edge_type_list

In [49]:
interactions

,Src,Dst,Weight,edge_type
0,0,GRN_Ligand,1,1
1,0,WNT4_Ligand,1,1
2,0,OXT_Ligand,1,1
3,0,SPX_Ligand,1,1
4,0,FAS_Ligand,1,1
...,...,...,...,...
528,6,CDK2_Receptor,1,1
529,6,ROBO3_Receptor,1,1
530,6,RECK_Receptor,1,1
531,6,SMO_Receptor,1,1


In [50]:
nodes.index = nodes["identifier"].tolist()

In [51]:
nodes = nodes.drop_duplicates("identifier")
nodes["Id"] = range(0,nodes.shape[0])

In [52]:
interactions["Src"] = nodes.loc[interactions["Src"].tolist()]["Id"].tolist()
interactions["Dst"] = nodes.loc[interactions["Dst"].tolist()]["Id"].tolist()

In [53]:
first_data,first_nodes,first_interactions = make_dataset(nodes,interactions,first=False,pathway_encode=False)

# Edit the ground truth to only contain information from this dataset

In [54]:
LR_nodes.index = LR_nodes["identifier"].tolist()

In [55]:
#LR_nodes["identifier"] = [row["identifier"] + "_" + row["category"] for index,row in LR_nodes.iterrows()]
LR_nodes = LR_nodes[(LR_nodes["identifier"].isin(ligand_list)) | (LR_nodes["identifier"].isin(receptor_list))]
Omnipath_network = Omnipath_network[(Omnipath_network["Src"].isin(LR_nodes["identifier"].tolist())) & (Omnipath_network["Dst"].isin(LR_nodes["identifier"].tolist()))]

In [56]:
ligand_list = Omnipath_network["Src"].tolist()
receptor_list = Omnipath_network["Dst"].tolist()
LR_nodes = LR_nodes[(LR_nodes["identifier"].isin(ligand_list)) | (LR_nodes["identifier"].isin(receptor_list))]
Omnipath_network = Omnipath_network[(Omnipath_network["Src"].isin(LR_nodes["identifier"].tolist())) & (Omnipath_network["Dst"].isin(LR_nodes["identifier"].tolist()))]

In [57]:
ligand_list = Omnipath_network["Src"].tolist()
receptor_list = Omnipath_network["Dst"].tolist()

In [58]:
LR_nodes["Id"] = range(0,LR_nodes.shape[0])
LR_nodes.index = LR_nodes["identifier"].tolist()
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()

In [59]:
interactions["edge_type"]=1

In [63]:
os.system("mkdir -p /data/GraphComm_Input/intercell_Drosophila")

0

In [65]:
nodes.to_csv("/data/GraphComm_Input/intercell_Drosophila/nodes.csv")
interactions.to_csv("/data/GraphComm_Input/intercell_Drosophila/interactions.csv")
meta.to_csv("/data/GraphComm_Input/intercell_Drosophila/meta.csv")
matrix.to_csv("/data/GraphComm_Input/intercell_Drosophila/matrix.csv")
LR_nodes.to_csv("/data/GraphComm_Input/intercell_Drosophila/LR_nodes.csv")
Omnipath_network.to_csv("/data/GraphComm_Input/intercell_Drosophila/Omnipath_network.csv")